In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/mixtral/pytorch/8x7b-v0.1/1/RELEASE
/kaggle/input/mixtral/pytorch/8x7b-v0.1/1/consolidated.00.pth
/kaggle/input/mixtral/pytorch/8x7b-v0.1/1/params.json
/kaggle/input/mixtral/pytorch/8x7b-v0.1/1/tokenizer.model
/kaggle/input/nbme-score-clinical-patient-notes/sample_submission.csv
/kaggle/input/nbme-score-clinical-patient-notes/patient_notes.csv
/kaggle/input/nbme-score-clinical-patient-notes/features.csv
/kaggle/input/nbme-score-clinical-patient-notes/train.csv
/kaggle/input/nbme-score-clinical-patient-notes/test.csv
/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1/config.json
/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1/pytorch_model-00002-of-00002.bin
/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1/tokenizer.json
/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1/tokenizer_config.json
/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1/pytorch_model.bin.index.json
/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1/pytorch_model-00001-of-00002.bin
/kaggle/

In [13]:
import pandas as pd
patient_notes = pd.read_csv("/kaggle/input/nbme-score-clinical-patient-notes/patient_notes.csv")
features = pd.read_csv("/kaggle/input/nbme-score-clinical-patient-notes/features.csv")
train = pd.read_csv("/kaggle/input/nbme-score-clinical-patient-notes/train.csv")


In [14]:
patient_notes.head() 


,pn_num,case_num,pn_history
0,0,0,"17-year-old male, has come to the student heal..."
1,1,0,17 yo male with recurrent palpitations for the...
2,2,0,Dillon Cleveland is a 17 y.o. male patient wit...
3,3,0,a 17 yo m c/o palpitation started 3 mos ago; \...
4,4,0,17yo male with no pmh here for evaluation of p...


In [17]:
patient_notes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42146 entries, 0 to 42145
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   pn_num      42146 non-null  int64 
 1   case_num    42146 non-null  int64 
 2   pn_history  42146 non-null  object
dtypes: int64(2), object(1)
memory usage: 987.9+ KB


In [15]:
features.head()

,feature_num,case_num,feature_text
0,0,0,Family-history-of-MI-OR-Family-history-of-myoc...
1,1,0,Family-history-of-thyroid-disorder
2,2,0,Chest-pressure
3,3,0,Intermittent-symptoms
4,4,0,Lightheaded


In [16]:
features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 143 entries, 0 to 142
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   feature_num   143 non-null    int64 
 1   case_num      143 non-null    int64 
 2   feature_text  143 non-null    object
dtypes: int64(2), object(1)
memory usage: 3.5+ KB


In [18]:
df_annotations = pd.merge(train, features, on=['case_num', 'feature_num'], how='left')

In [21]:
df_annotations["location"]

0                 ['696 724']
1                 ['668 693']
2                 ['203 217']
3        ['70 91', '176 183']
4                 ['222 258']
                 ...         
14295                      []
14296                      []
14297             ['274 282']
14298             ['421 437']
14299             ['314 330']
Name: location, Length: 14300, dtype: object

In [22]:
import ast

In [23]:
def process_location_string(loc_str):
    try:
        locations = ast.literal_eval(loc_str)  # Try converting to a list

        # Convert each span to a tuple of integers
        return [(int(start), int(end)) for start, end in locations]  
    except (ValueError, SyntaxError):
        # Handle cases where the string is not a valid list format
        return []  # Or you could raise an error for better debugging 


In [24]:
df_annotations["location"] = df_annotations["location"].apply(process_location_string)


In [26]:
def combine_notes_features(row):
    notes = patient_notes[patient_notes['pn_num'] == row['pn_num']]['pn_history'].iloc[0]
    annotations = row['annotation'].split('|')  # Split annotations on '|'
    locations = row['location']  # Locations is already a list

    # Logic to insert annotations into the notes based on their 'location' 
    for annotation, loc in zip(annotations, locations):  
        start, end = loc  # Since loc is now a tuple (start, end)
        notes = notes[:start] + f"[{annotation}]" + notes[end:]

    return notes

df_annotations['combined_note'] = df_annotations.apply(combine_notes_features, axis=1)


In [30]:
df_annotations.head()

,id,case_num,pn_num,feature_num,annotation,location,feature_text,combined_note
0,00016_000,0,16,0,['dad with recent heart attcak'],[],Family-history-of-MI-OR-Family-history-of-myoc...,HPI: 17yo M presents with palpitations. Patien...
1,00016_001,0,16,1,"['mom with ""thyroid disease']",[],Family-history-of-thyroid-disorder,HPI: 17yo M presents with palpitations. Patien...
2,00016_002,0,16,2,['chest pressure'],[],Chest-pressure,HPI: 17yo M presents with palpitations. Patien...
3,00016_003,0,16,3,"['intermittent episodes', 'episode']",[],Intermittent-symptoms,HPI: 17yo M presents with palpitations. Patien...
4,00016_004,0,16,4,['felt as if he were going to pass out'],[],Lightheaded,HPI: 17yo M presents with palpitations. Patien...


In [33]:
df_annotations.to_csv('/kaggle/working/combined.csv')

In [37]:
df_annotations.dropna(subset=['feature_text'], inplace=True)  

In [40]:
df_annotations.dropna(subset=['combined_note'], inplace=True)  

In [41]:
df_annotations = df_annotations.drop('location', axis=1)  


In [42]:
df_annotations

,id,case_num,pn_num,feature_num,annotation,feature_text,combined_note
0,00016_000,0,16,0,['dad with recent heart attcak'],Family-history-of-MI-OR-Family-history-of-myoc...,HPI: 17yo M presents with palpitations. Patien...
1,00016_001,0,16,1,"['mom with ""thyroid disease']",Family-history-of-thyroid-disorder,HPI: 17yo M presents with palpitations. Patien...
2,00016_002,0,16,2,['chest pressure'],Chest-pressure,HPI: 17yo M presents with palpitations. Patien...
3,00016_003,0,16,3,"['intermittent episodes', 'episode']",Intermittent-symptoms,HPI: 17yo M presents with palpitations. Patien...
4,00016_004,0,16,4,['felt as if he were going to pass out'],Lightheaded,HPI: 17yo M presents with palpitations. Patien...
...,...,...,...,...,...,...,...
14295,95333_912,9,95333,912,[],Family-history-of-migraines,Stephanie madden is a 20 year old woman compla...
14296,95333_913,9,95333,913,[],Female,Stephanie madden is a 20 year old woman compla...
14297,95333_914,9,95333,914,['photobia'],Photophobia,Stephanie madden is a 20 year old woman compla...
14298,95333_915,9,95333,915,['no sick contacts'],No-known-illness-contacts,Stephanie madden is a 20 year old woman compla...


In [43]:
df_annotations.to_csv('/kaggle/working/combined.csv')

In [44]:

data_array = df_annotations.to_numpy()  # Convert the DataFrame to a NumPy array


In [46]:
np.save('/kaggle/working/combined_features_patient_notes.npy', data_array) 

In [1]:
import numpy as np

In [5]:
x = np.load("../Data_FineTune/combined_features_patient_notes.npy", allow_pickle=True)

In [7]:
x[0]

array(['00016_000', 0, 16, 0, "['dad with recent heart attcak']",
       'Family-history-of-MI-OR-Family-history-of-myocardial-infarction',
       'HPI: 17yo M presents with palpitations. Patient reports 3-4 months of intermittent episodes of "heart beating/pounding out of my chest." 2 days ago during a soccer game had an episode, but this time had chest pressure and felt as if he were going to pass out (did not lose conciousness). Of note patient endorses abusing adderall, primarily to study (1-3 times per week). Before recent soccer game, took adderrall night before and morning of game. Denies shortness of breath, diaphoresis, fevers, chills, headache, fatigue, changes in sleep, changes in vision/hearing, abdominal paun, changes in bowel or urinary habits. \r\nPMHx: none\r\nRx: uses friends adderrall\r\nFHx: mom with "thyroid disease," dad with recent heart attcak\r\nAll: none\r\nImmunizations: up to date\r\nSHx: Freshmen in college. Endorses 3-4 drinks 3 nights / week (on weekends),